# Building a Project with the ChatGPT API

## Generating an Image using DALL-E

## 06_04 Solution - Generate Images Using Image Generation

### In this hands-on challenge, you'll use the website summary to generate an image using the Images API.

		A sample flow is shown below:

		Set the context using a system message.
        
        System: You are a friendly AI assistant that helps compose professional-sounding tweets 
                for Twitter that often go viral based on a website I provide. You will provide 
                a summary of the website in 30 words or less.
		
		Assistant: Sure, what's the website you want me to summarize for you?
        
        User: http://www.amazon.com/sagemaker
		
		Assistant: Amazon SageMaker is a fully-managed service that enables developers and data 
        scientists to quickly and easily build, train, and deploy machine learning models at any 
        scale. It provides a range of tools and features to streamline the machine learning workflow, 
        including data labeling, model training, and deployment.
		
		User: Can you optimize the summary in 10 words in order to generate an image?

        Assistant:  Amazon SageMaker: Build, train, and deploy ML models easily.
        
### The summary returned is then passed to DALL-E

### Import the libraries and enviornment file to gain access to the Open API Key
#### The key can be generated here: https://platform.openai.com/account/api-keys

In [1]:
import os
import openai #needed for error handling
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

### Authenticate to the API using the API Key
#### Pull from environment variables or use openai.api_key = ("your_key_here") to hardcode the key

In [2]:
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY']  
)

In [3]:
TEXT_MODEL = "gpt-3.5-turbo"

#sets the persona for the AI assistant using a system message
context = [{'role':'system', 'content':"""You are a friendly AI assistant that 
                                              helps compose professional-sounding tweets 
                                              for Twitter that often go viral based on a 
                                              website I provide. You will provide a summary 
                                              of the website in 50 words or less."""
            }]  

In [4]:
# Each interation with the AI assistant is a new session so the entire chat/message history, 
# including user prompts and assistant responses must be included in each exchange with the
# model/assistant so that it "remembers"

def collect_messages(role, message): #keeps track of the message exchange between user and assistant
    context.append({'role': role, 'content':f"{message}"})

In [5]:
# Sends the prompts to the model for a completion/response

def get_completion(temperature=0): 
    try:
        response = client.chat.completions.create(
            model=TEXT_MODEL,
            messages=context, 
            temperature=temperature, 
        )

        print("\n Assistant: ", response.choices[0].message.content, "\n")

        return response.choices[0].message.content
    except openai.APIError as e:
        print(e.http_status)
        print(e.error)
        return e.error

In [6]:
# Create images from scratch based on the website summary

def generate_image(summary):
    print(summary)
    
    try:
        response = client.images.generate(
          model="dall-e-3",
          prompt=summary,
          size="1024x1024",
          quality="standard",
          n=1, #select the number of images you want generated
        )
        
        image_url = response.data[0].url #URLs will expire after an hour

        return image_url
    except openai.APIError as e:
        print(e.http_status)
        print(e.error)
        return e.error

In [7]:
#Start the conversation between the user and the AI assistant/chatbot
    
while True:
    summary = get_completion()
    
    collect_messages('assistant', summary) #stores the response from the AI assistant
    
    user_prompt = input('User: ') #input box for entering prompt
    
    if user_prompt == 'exit': #end the conversation with the AI assistant
        print("\n Goodbye")
        break

    collect_messages('user', user_prompt) #stores the user prompt


 Assistant:  Sure, please provide me with the website you would like me to summarize. 

User: http://www.amazon.com/sagemaker

 Assistant:  Amazon SageMaker is a comprehensive machine learning service provided by Amazon Web Services (AWS). It allows users to build, train, and deploy machine learning models at scale. With a wide range of tools and capabilities, SageMaker simplifies the process of developing and deploying machine learning models for various applications. 

User: Can you optimize the summary in 10 words in order to generate an image?

 Assistant:  Amazon SageMaker: AWS machine learning service for building, training, deploying models. 

User: exit

 Goodbye


In [8]:
url = generate_image(summary)

Amazon SageMaker: AWS machine learning service for building, training, deploying models.


In [9]:
from IPython.display import Image
Image(url=url)